# Homework 3

Please refer to problems on Piazza as HW3.8 etc.  Points may not be awarded for correct numerical answers without working code.

## Questions 1-3

This problem is related to the self-assessment question in the lesson titled "Unbalanced Transportation Problem" and is modeled after a problem found in Practical Management Science by Wayne L. Winston and S. Christian Albright.  

International Textile Company, Ltd, Kong–based firm that distributes textiles world- wide. They have mills for weaving bolts of cotton fabric in the Bahamas, Hong Kong, Korea, Nigeria, and Venezuela.  The mills ship bolts of cotton to distribution eight distribution centers (listed below).

The company is owned by the Lao family. Present plans are to remain in Hong Kong through the transition in governments. Should the People’s Republic of China continue its economic renaissance, the company hopes to use its current base to expand operations to the mainland. International Textile has mills in the Bahamas, Hong Kong, Korea, Nigeria, and Venezuela, each weaving fabrics out of two or more raw fibers: cotton, polyester, and/or silk. The mills service eight company distribution centers.  Cotton supplies and demands at the distribution center are shown below.  Shipping costs depend on both distances and other factors and are in the cell below.

Additionally, the company is switching to a different shipping method with a limit of 500 cotton bolts on each route.  Moreover, shipments from Venezuela to the United States will no longer be possible (no shipments to Los Angeles, Chicago, and New York).  **Note that you'll have to set up this constraint yourself.  We haven't provided data or code for it.**


### Question 1 <font color="magenta">(10 points, manual)</font>
This transportation problem is unbalanced.  Write Pyomo code to find the minimum shipping cost.  Solve the problem without introducing any dummy mills or distribution centers.  Include your complete, abstract, Pyomo code in the cell below and in your CoCalc notebook.  Your code should output a data frame that displays the shipping amounts.  Here is some problem data to get you started:
```python
mills = ['Bahamas', 'Hong Kong', 'Korea', 'Nigeria', 'Venezuela']
dist_ctrs = [
    'Los Angeles', 'Chicago', 'London', 'Mexico City', 'Manila', 'Rome',
    'Tokyo', 'New York'
]
supply = [1000, 2000, 1000, 2000, 1000]
demand = [500, 800, 900, 900, 800, 100, 200, 700]
ship_cost = [[2, 2, 3, 3, 7, 4, 7, 1],
             [6, 7, 8, 10, 2, 9, 4, 8],
             [5, 6, 8, 11, 4, 9, 1, 7],
             [14, 12, 6, 9, 11, 7, 5, 10],
             [4, 3, 5, 1, 9, 6, 11, 4]]
# you'll need to set up data for the infeasible routes!
```

In [1]:
from pyomo.environ import *
import numpy as np
import pandas as pd

mills = ['Bahamas', 'Hong Kong', 'Korea', 'Nigeria', 'Venezuela']
dist_ctrs = [
    'Los Angeles', 'Chicago', 'London', 'Mexico City', 'Manila', 'Rome',
    'Tokyo', 'New York'
]
sup = [1000, 2000, 1000, 2000, 1000]
dem = [500, 800, 900, 900, 800, 100, 200, 700]
bigM = 1000
ship_cost = [[2, 2, 3, 3, 7, 4, 7, 1],
             [6, 7, 8, 10, 2, 9, 4, 8],
             [5, 6, 8, 11, 4, 9, 1, 7],
             [14, 12, 6, 9, 11, 7, 5, 10],
             [bigM, bigM, 5, 1, 9, 6, 11, bigM]]

max_capacity = 500

supply = dict(zip(mills, sup))
demand = dict(zip(dist_ctrs, dem))
unit_ship_cost = { m : { d : ship_cost[i][j] for j,d in enumerate(dist_ctrs) } for i,m in enumerate(mills) }

#supply > demand: therefore supply constraint needs to change to inequality

# Declaration
model = ConcreteModel()

# Decision Variables
model.transp = Var(mills, dist_ctrs, domain=NonNegativeReals)

# Objective
model.total_cost = Objective(expr=sum(unit_ship_cost[m][d] * model.transp[m, d]
                                      for m in mills for d in dist_ctrs),
                             sense=minimize)

# Constraints
model.supply_ct = ConstraintList()
for m in mills:
    model.supply_ct.add(
        sum(model.transp[m, d] for d in dist_ctrs) <= supply[m])

model.demand_ct = ConstraintList()
for d in dist_ctrs:
    model.demand_ct.add(
        sum(model.transp[m, d] for m in mills) == demand[d])

model.capacity_ct = ConstraintList()
for m in mills:
    for d in dist_ctrs:
        model.capacity_ct.add(model.transp[m, d] <= max_capacity)

### SOLUTION ###
solver = SolverFactory('glpk')
solver.solve(model)

### OUTPUT ###
print(f"Minimum Total Cost = ${model.total_cost():,.2f}")

# dataframes are displayed nicely in Jupyter
dvars = pd.DataFrame([[model.transp[m, d]() for d in dist_ctrs]
                      for m in mills],
                     index=mills,
                     columns=dist_ctrs)
print("Number to ship from each mill to each distribution center:")
dvars

Minimum Total Cost = $19,400.00
Number to ship from each mill to each distribution center:


,Los Angeles,Chicago,London,Mexico City,Manila,Rome,Tokyo,New York
Bahamas,0.0,100.0,0.0,400.0,0.0,0.0,0.0,500.0
Hong Kong,500.0,200.0,0.0,0.0,500.0,0.0,0.0,200.0
Korea,0.0,500.0,0.0,0.0,300.0,0.0,200.0,0.0
Nigeria,0.0,0.0,400.0,0.0,0.0,100.0,0.0,0.0
Venezuela,0.0,0.0,500.0,500.0,0.0,0.0,0.0,0.0


### Question 2 <font color="magenta">(1 point, auto)</font>

What is the minimum total shipping cost?  Enter your answer to the nearest dollar.

**$19,400**

### Question 3 <font color="magenta">(1 point, auto)</font>

To achieve the minimum total shipping cost, how many bolts should be shipped from Korea to Manila?

**300**


## Questions 4-6

Each of three warehouses has two products, pA and pB, in stock to send to five different stores.  Each route from a warehouse to a store has a limited capacity of 700 products (total of pA and pB) or there is no shipping available on that route.  The unit shipping cost on each route doesn't depend on the product.  For example, it costs $14/unit to ship product pA or pB from warehouse wA to store sA.  Supply, demand, costs, and capacities, are loaded into data frames below for your convenience.  To access values in the data frame:  `cost.loc['wA','sC']` gives 12.

Review the material in the lesson about multiple products as needed.

Here is the problem data to get you started (you'll need to import pandas as pd):
```python
warehouses = ['wA','wB','wC']
stores = ['sA','sB','sC','sD','sE']
products = ['pA','pB']
supply = pd.DataFrame([ [400,100,500], [500,100,400]], 
                         index = products, columns = warehouses)
demand = pd.DataFrame([ [200,100,250,100,300], [200,100,150,100,400]],
                         index = products, columns = stores)
cost = pd.DataFrame( [ [14, 11, 12, 5000, 5000],
                         [5000, 15, 18, 21, 5000],
                         [5000,5000,9,10,12] ],
                       index = warehouses,columns = stores)
capacity = pd.DataFrame( [ [700, 700, 700, 0, 0],
                           [0, 700, 700, 700, 0],
                           [0, 0, 700, 700, 700] ],
                       index = warehouses, columns = stores)
```

### Question 4 <font color="magenta">(12 points, manual)</font>

Include your Pyomo code for minimizing the total shipping cost below.  Your code should also be complete and executed in your CoCalc notebook.

In [2]:
from pyomo.environ import *
import pandas as pd

warehouses = ['wA','wB','wC']
stores = ['sA','sB','sC','sD','sE']
products = ['pA','pB']
supply = pd.DataFrame([ [400,100,500], [500,100,400]], 
                         index = products, columns = warehouses)
demand = pd.DataFrame([ [200,100,250,100,300], [200,100,150,100,400]],
                         index = products, columns = stores)
cost = pd.DataFrame( [ [14, 11, 12, 5000, 5000],
                       [5000, 15, 18, 21, 5000],
                       [5000,5000,9,10,12] ],
                       index = warehouses,columns = stores)
capacity = pd.DataFrame( [ [700, 700, 700, 0, 0],
                           [0, 700, 700, 700, 0],
                           [0, 0, 700, 700, 700] ],
                       index = warehouses, columns = stores)

# supply > demand, so this constraint must be adjusted
# Declaration
model = ConcreteModel()

# Decision Variables
model.transp = Var(products, warehouses, stores, domain=NonNegativeReals)

# Objective
model.total_cost = Objective(expr=sum(cost.loc[w,s] * sum(model.transp[p,w,s]
                                      for p in products) for w in warehouses for s in stores),
                                      sense=minimize)

# constraints
model.capacity_ct = ConstraintList()
for w in warehouses:
    for s in stores:
        model.capacity_ct.add(sum(model.transp[p,w,s] for p in products) <= capacity.loc[w,s])

model.supply_ct = ConstraintList()
for p in products:
    for w in warehouses:
        model.supply_ct.add(sum(model.transp[p,w,s] for s in stores) <= supply.loc[p,w])

model.demand_ct = ConstraintList()
for p in products:
    for s in stores:
        model.demand_ct.add(sum(model.transp[p,w,s] for w in warehouses) == demand.loc[p,s])

### SOLUTION ###
solver = SolverFactory('glpk')
solver.solve(model)

### OUTPUT ###
print(f"Minimum Total Cost = ${model.total_cost():,.2f}")
print('\n')

for p in products:
    dvars = pd.DataFrame([[model.transp[p,w,s]() for s in stores]
                      for w in warehouses],
                     index=warehouses,
                     columns=stores)
    print(f'Amount of {p} to ship:')
    print(dvars.to_markdown())
    print('\n')

Minimum Total Cost = $24,000.00


Amount of pA to ship:
|    |   sA |   sB |   sC |   sD |   sE |
|:---|-----:|-----:|-----:|-----:|-----:|
| wA |  200 |   50 |  150 |    0 |    0 |
| wB |    0 |   50 |    0 |    0 |    0 |
| wC |    0 |    0 |  100 |  100 |  300 |


Amount of pB to ship:
|    |   sA |   sB |   sC |   sD |   sE |
|:---|-----:|-----:|-----:|-----:|-----:|
| wA |  200 |  100 |  150 |    0 |    0 |
| wB |    0 |    0 |    0 |  100 |    0 |
| wC |    0 |    0 |    0 |    0 |  400 |




### Question 5 <font color="magenta">(1 point, auto)</font>

What is the minimum total shipping cost?  Enter your answer to the nearest dollar.

**$24,000**

### Question 6 <font color="magenta">(1 point, auto)</font>

To achieve the minimum total shipping cost how many of product A should be shipped from warehouse C to store C?

**100**

## Questions 7-9

The coach of an age group swim team needs to assign swimmers to a 200-yard medley relay team to send to the Junior Olympics. Since most of his best swimmers are very fast in more than one stroke, it is not clear which swimmer should be assigned to each of the four strokes. The five fastest swimmers and the best times (in seconds) they have achieved in each of the strokes (for 50 yards) are

<img src = "images/swim.png" width="600">

The coach wishes to determine how to assign four swimmers to the four different strokes to minimize the sum of the corresponding best times.  

### Question 7 <font color="magenta">(10 points, manual)</font>
Treat this as an assignment problem and use Pyomo to find the optimal solution.  For full credit you must use an abstract approach to the solution code and display nicely formatted output.  Include complete, executed code in your CoCalc notebook.

In [3]:
from pyomo.environ import *
import numpy as np
import pandas as pd

strokes = ['backstroke', 'breaststroke', 'butterfly', 'freestyle', 'NaN']
demand = dict(zip(strokes, [1,1,1,1,1]))

swimmers = ['Carl', 'Chris', 'David', 'Tony', 'Ken']
supply = dict(zip(swimmers, [1,1,1,1,1]))
stroke_times = [[37.7, 32.9, 35.4, 37, 33.7],
                [43.4, 33.1, 42.2, 34.5, 41.8],
                [33.3, 28.5, 38.9, 30.4, 33.6],
                [29.2, 26.4, 29.6, 28.5, 31.1],
                [0, 0, 0, 0, 0]]

time_dict = { st : { sw: stroke_times[i][j] for j, sw in enumerate(swimmers)} for i, st in enumerate(strokes)}

model = ConcreteModel()

model.assign = Var(strokes, swimmers, domain=NonNegativeReals)

model.total_time = Objective(expr=sum(time_dict[st][sw] * model.assign[st, sw] 
                                      for st in strokes for sw in swimmers), sense=minimize)

model.demand_ct = ConstraintList()
for st in strokes:
    model.demand_ct.add(
        sum(model.assign[st, sw] for sw in swimmers) == demand[st])

model.supply_ct = ConstraintList()
for sw in swimmers:
    model.supply_ct.add(
        sum(model.assign[st, sw] for st in strokes) == supply[sw])

solver = SolverFactory('glpk')
solver.solve(model)

# display solution
print(f"Minimum time = {model.total_time():,.1f}s")

# put amounts in dataframe for nicer display
dvars = pd.DataFrame([[model.assign[st, sw]() for sw in swimmers]
                      for st in strokes],
                     index = strokes,
                     columns=swimmers)
print("Swimmer assignments to strokes:")
dvars

Minimum time = 125.9s
Swimmer assignments to strokes:


,Carl,Chris,David,Tony,Ken
backstroke,0.0,0.0,0.0,0.0,1.0
breaststroke,0.0,0.0,0.0,1.0,0.0
butterfly,0.0,1.0,0.0,0.0,0.0
freestyle,1.0,0.0,0.0,0.0,0.0
NaN,0.0,0.0,1.0,0.0,0.0


### Question 8 <font color="magenta">(1 point, auto)</font>

What is the minimum total race time?  Enter your answer to the nearest tenth of a second.

**125.9**

### Question 9 <font color="magenta">(1 point, auto)</font>

Which swimmer is assigned to swim backstroke?

**Ken**


## Questions 10-12

Reliable Construction wants to determine when each activity should start in a construction schedule in order to minimize the overall time it takes to construct a large commercial building.  The tasks, order of tasks, and their durations in days are specified in the table below. 

Remember that we need to append a final task that has all other tasks as immediate predecessors and an estimated duration of zero days.

| Activity  | Description  | Immediate Predecessors  |  Estimated Duration |
|---|---|---|---|
| A  | Excavate  |  -- |  14 |
| B  | Lay the Foundation  | A  | 21  |
| C  |  Put up the rough wall |  B |  63 |
| D  |  Put up the roof | C  |  35 |
| E  |  Install the exterior plumbing | C  |  42 |
| F  |  Install the interior plumbing | E  |  56 |
| G  |  Put up the exterior siding | D  | 49  |
| H  |  Do the exterior painting | E,G  | 63  |
| I  |  Do the electrical work | C  | 28  |
| J  |  Put up the wallboard |  F,I |  35 |
| K  |  Install the flooring |  J | 14  |
| L  |  Do the interior painting |  J | 35  |
| M  |  Install the exterior fixtures | H  | 14  |
| N  |  Install the interior fixtures | K,L  | 35  |

Note that the precedent constraints specified in the table are presented a bit differently in the lesson, so you'll need to adjust for that somehow.

### Question 10 <font color="magenta">(10 points, manual)</font>

Write abstract, generalizable Pyomo code to solve this scheduling problem

Include your code below and in your CoCalc notebook.

In [4]:
from pyomo.environ import *
import numpy as np
import pandas as pd

pred_dict = {
    'B':['A'],
    'C':['B'],
    'D':['C'],
    'E':['C'],
    'F':['E'],
    'G':['D'],
    'H':['E','G'],
    'I':['C'],
    'J':['F','I'],
    'K':['J'],
    'L':['J'],
    'M':['H'],
    'N':['K','L'],
    'final':['A','B','C','D','E','F','G','H','I','J','K','L','M','N']
}

descriptions = ['excavation', 'laying foundation', 'rough wall',
               'roofing', 'exterior plumbing', 'interior plumbing',
               'exterior siding', 'exterior painting', 'electrical work', 'wallboard', 'flooring',
               'interior painting', 'exterior fixtures', 'interior fixtures', 'final']
tasks = ['A', *pred_dict]
task_dict = dict(zip(tasks, descriptions))
durations = [14, 21, 63, 35, 42, 56, 49, 63, 28, 35, 14, 35, 14, 35, 0]
task_duration_dict = dict(zip(tasks, durations))

#Model
M = ConcreteModel(name = "Schedule")

M.start = Var(tasks, domain = NonNegativeReals)

M.length = Objective( expr = M.start['final'], sense = minimize )

M.pred_ct = ConstraintList()
for after in pred_dict.keys():
    for before in pred_dict[after]:
        M.pred_ct.add( M.start[after] - task_duration_dict[before] >= M.start[before])
        
### Solution ###
solver = SolverFactory('glpk')
solver.solve(M)

### Display ###
print(f"Total Time = {M.length():,.0f}\n")
for t in tasks:
    print(f"Start {task_dict[t]} at {M.start[t]():.0f} and end at {M.start[t]()+task_duration_dict[t]:.0f}")

Total Time = 301

Start excavation at 0 and end at 14
Start laying foundation at 14 and end at 35
Start rough wall at 35 and end at 98
Start roofing at 98 and end at 133
Start exterior plumbing at 98 and end at 140
Start interior plumbing at 140 and end at 196
Start exterior siding at 133 and end at 182
Start exterior painting at 182 and end at 245
Start electrical work at 98 and end at 126
Start wallboard at 196 and end at 231
Start flooring at 231 and end at 245
Start interior painting at 231 and end at 266
Start exterior fixtures at 245 and end at 259
Start interior fixtures at 266 and end at 301
Start final at 301 and end at 301


### Question 11 <font color="magenta">(1 point, auto)</font>

What is the minimum total construction time?  Enter your answer to the nearest day.

**301**

### Question 12 <font color="magenta">(1 point, auto)</font>

On what day should Interior Plumbing start to achieve the minimum construction time?

**140**